#### Restart and Run All

In [1]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
#data_path = "\\users\\user\\onedrive\\a7\\Data"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 9, 1)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-09-01
yesterday: 2023-08-31


In [3]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [4]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1308177,STA,2023-08-31,2023-08-31,0,14.80,14.60,-0.20,-1.35%,-2,0,14.80,13.20,"77,307,636",2023-08-31,2023-08-31,1


In [5]:
names = tmp["name"]
type(names)

pandas.core.series.Series

In [6]:
name = names.to_string(index=False)
type(name)

str

In [7]:
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'STA'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,123,STA,0.00,0.00,B,12.70,0.00,0.00,0.00,40.00,"2,500",-4,4,"5,000",0.000000,0.000000,RD70pct,SET100


### End of Tables in the process

In [8]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2023-08-31'
ORDER BY a.name



,name
0,AH
1,AIMIRT
2,ASK
3,ASP
4,AWC
5,BBL
6,BCH
7,BEM
8,BH
9,CK


In [28]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2023-08-31' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [29]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,DIF,2023-08-30,2023-08-31,9.30,9.45,"70,056,756",9.45,9.15,B,SET,0.15,1.61%
1,JMART,2023-08-30,2023-08-31,22.20,24.00,"173,996,992",24.00,21.20,B,SET50,1.80,8.11%
2,NER,2023-08-31,2023-08-31,4.82,4.80,"162,020,712",4.82,4.56,B,SET,-0.02,-0.41%
3,SENA,2023-08-25,2023-08-31,2.82,2.90,"19,557,312",2.90,2.78,B,SET,0.08,2.84%
4,SINGER,2023-08-25,2023-08-31,11.30,14.90,"892,953,120",14.90,10.30,B,SET100,3.60,31.86%
5,STA,2023-08-31,2023-08-31,14.80,14.60,"77,307,636",14.80,13.20,B,SET100,-0.20,-1.35%
6,SYNEX,2023-08-30,2023-08-31,13.00,13.70,"41,373,116",13.70,11.60,B,SET,0.70,5.38%
7,WHART,2023-08-28,2023-08-31,10.40,10.70,"18,912,464",10.70,10.40,B,SET,0.30,2.88%
8,ASK,2023-08-30,2023-08-31,25.25,26.00,"12,773,172",26.00,23.70,I,SET,0.75,2.97%
9,ASP,2023-08-30,2023-08-31,2.94,2.92,"45,424,780",3.02,2.80,I,SET,-0.02,-0.68%


In [11]:
df.shape

(48, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [12]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,AH,2023-08-30,2023-08-31,34.50,36.00,"11,071,484",36.00,34.00,4.35%,B
1,AWC,2023-08-31,2023-08-31,4.42,4.36,"386,017,008",4.42,4.06,-1.36%,B
2,DIF,2023-08-30,2023-08-31,9.30,9.45,"70,056,756",9.45,9.15,1.61%,B
3,JASIF,2023-08-28,2023-08-31,6.95,6.80,"196,650,488",7.00,6.80,-2.16%,B
4,JMART,2023-08-30,2023-08-31,22.20,24.00,"173,996,992",24.00,21.20,8.11%,B
5,JMT,2023-08-31,2023-08-31,47.75,46.50,"25,272,886",47.75,45.50,-2.62%,B
6,MCS,2023-08-31,2023-08-31,7.00,6.80,"2,085,184",7.00,6.00,-2.86%,B
7,NER,2023-08-31,2023-08-31,4.82,4.80,"162,020,712",4.82,4.56,-0.41%,B
8,ORI,2023-08-31,2023-08-31,10.80,10.60,"30,032,562",10.80,10.10,-1.85%,B
9,PTTGC,2023-08-31,2023-08-31,37.75,37.25,"74,106,528",38.00,35.75,-1.32%,B


In [13]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(one_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [14]:
df[cols].shape

(48, 10)

In [15]:
sales = df[cols]
sales

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,AH,2023-08-30,2023-08-31,34.50,36.00,11071484,36.00,34.00,4.35,B
1,AWC,2023-08-31,2023-08-31,4.42,4.36,386017008,4.42,4.06,-1.36,B
2,DIF,2023-08-30,2023-08-31,9.30,9.45,70056756,9.45,9.15,1.61,B
3,JASIF,2023-08-28,2023-08-31,6.95,6.80,196650488,7.00,6.80,-2.16,B
4,JMART,2023-08-30,2023-08-31,22.20,24.00,173996992,24.00,21.20,8.11,B
5,JMT,2023-08-31,2023-08-31,47.75,46.50,25272886,47.75,45.50,-2.62,B
6,MCS,2023-08-31,2023-08-31,7.00,6.80,2085184,7.00,6.00,-2.86,B
7,NER,2023-08-31,2023-08-31,4.82,4.80,162020712,4.82,4.56,-0.41,B
8,ORI,2023-08-31,2023-08-31,10.80,10.60,30032562,10.80,10.10,-1.85,B
9,PTTGC,2023-08-31,2023-08-31,37.75,37.25,74106528,38.00,35.75,-1.32,B


In [16]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
prices = pd.read_csv(input_file)
prices

,name,date,price,change,%change,open,high,low,volume,value(m)
0,ACE,2023-09-01,1.75,-0.02,-1.129944,1.77,1.81,1.75,18097810,31995.08872
1,ADVANC,2023-09-01,215.00,-1.00,-0.462963,216.00,217.00,214.00,3496605,752528.99700
2,AEONTS,2023-09-01,173.50,-1.50,-0.857143,176.00,176.50,172.50,248725,43308.93450
3,AH,2023-09-01,36.00,0.00,0.000000,36.25,36.50,35.50,1973637,70954.68825
4,AIMIRT,2023-09-01,10.80,-0.10,-0.917431,10.90,10.90,10.80,25701,280.04060
...,...,...,...,...,...,...,...,...,...,...
185,WHAIR,2023-09-01,7.00,-0.05,-0.709220,7.05,7.05,7.00,159005,1116.32510
186,WHART,2023-09-01,10.60,-0.10,-0.934579,10.70,10.70,10.50,165509,1750.92630
187,WHAUP,2023-09-01,3.92,-0.04,-1.010101,3.94,3.94,3.90,509221,1990.76658
188,WICE,2023-09-01,8.45,0.10,1.197605,8.40,8.60,8.40,1215712,10349.19760


In [17]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,date,price,change,%change,open,high,low,volume,value(m)
0,AH,2023-08-30,2023-08-31,34.50,36.00,11071484,36.00,34.00,4.35,B,2023-09-01,36.00,0.00,0.000000,36.25,36.50,35.50,1973637,7.095469e+04
1,AWC,2023-08-31,2023-08-31,4.42,4.36,386017008,4.42,4.06,-1.36,B,2023-09-01,4.26,-0.10,-2.293578,4.36,4.38,4.24,64640345,2.769404e+05
2,DIF,2023-08-30,2023-08-31,9.30,9.45,70056756,9.45,9.15,1.61,B,2023-09-01,9.35,-0.10,-1.058201,9.45,9.45,9.35,3240715,3.045084e+04
3,JASIF,2023-08-28,2023-08-31,6.95,6.80,196650488,7.00,6.80,-2.16,B,2023-09-01,6.75,-0.05,-0.735294,6.80,6.80,6.75,2217254,1.498729e+04
4,JMART,2023-08-30,2023-08-31,22.20,24.00,173996992,24.00,21.20,8.11,B,2023-09-01,23.20,-0.80,-3.333333,23.90,24.20,23.00,29156182,6.848510e+05
5,JMT,2023-08-31,2023-08-31,47.75,46.50,25272886,47.75,45.50,-2.62,B,2023-09-01,46.25,-0.25,-0.537634,47.00,47.50,46.00,14292072,6.672382e+05
6,MCS,2023-08-31,2023-08-31,7.00,6.80,2085184,7.00,6.00,-2.86,B,2023-09-01,6.80,0.00,0.000000,7.05,7.05,6.75,294423,2.013717e+03
7,NER,2023-08-31,2023-08-31,4.82,4.80,162020712,4.82,4.56,-0.41,B,2023-09-01,4.92,0.12,2.500000,4.86,4.98,4.86,17620913,8.667607e+04
8,ORI,2023-08-31,2023-08-31,10.80,10.60,30032562,10.80,10.10,-1.85,B,2023-09-01,10.50,-0.10,-0.943396,10.60,10.70,10.50,4301496,4.568359e+04
9,PTTGC,2023-08-31,2023-08-31,37.75,37.25,74106528,38.00,35.75,-1.32,B,2023-09-01,37.00,-0.25,-0.671141,37.25,37.50,37.00,4915019,1.828963e+05


In [18]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
colu

['name',
 'fm_date',
 'to_date',
 'fm_price',
 'to_price',
 'qty',
 'max_price',
 'min_price',
 'percent',
 'status',
 'price',
 'change',
 'volume',
 'date']

In [19]:
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,AH,2023-08-30,2023-08-31,34.50,36.00,11071484,36.00,34.00,4.35,B,36.00,0.00,1973637,2023-09-01
1,AWC,2023-08-31,2023-08-31,4.42,4.36,386017008,4.42,4.06,-1.36,B,4.26,-0.10,64640345,2023-09-01
2,DIF,2023-08-30,2023-08-31,9.30,9.45,70056756,9.45,9.15,1.61,B,9.35,-0.10,3240715,2023-09-01
3,JASIF,2023-08-28,2023-08-31,6.95,6.80,196650488,7.00,6.80,-2.16,B,6.75,-0.05,2217254,2023-09-01
4,JMART,2023-08-30,2023-08-31,22.20,24.00,173996992,24.00,21.20,8.11,B,23.20,-0.80,29156182,2023-09-01
5,JMT,2023-08-31,2023-08-31,47.75,46.50,25272886,47.75,45.50,-2.62,B,46.25,-0.25,14292072,2023-09-01
6,MCS,2023-08-31,2023-08-31,7.00,6.80,2085184,7.00,6.00,-2.86,B,6.80,0.00,294423,2023-09-01
7,NER,2023-08-31,2023-08-31,4.82,4.80,162020712,4.82,4.56,-0.41,B,4.92,0.12,17620913,2023-09-01
8,ORI,2023-08-31,2023-08-31,10.80,10.60,30032562,10.80,10.10,-1.85,B,10.50,-0.10,4301496,2023-09-01
9,PTTGC,2023-08-31,2023-08-31,37.75,37.25,74106528,38.00,35.75,-1.32,B,37.00,-0.25,4915019,2023-09-01


In [20]:
df_merge[colu].shape

(48, 14)

In [21]:
df_merge.shape

(48, 19)

In [22]:
file_name = "daily-sales-prices.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(data_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(one_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

In [23]:
### Call ruby ruby\daily-out-new.rb

In [24]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [25]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [26]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AH       2023-08-30 2023-09-01  34.50  36.00   4.35%  11,071,484  36.00  34.00 B Update
AIMIRT   2023-08-25 2023-09-01  11.00  10.80  -1.82%   2,430,432  11.00  10.80 I Update
ASK      2023-09-01 2023-09-01  26.00  25.25  -2.88%  12,773,172  26.00  23.70 I Insert
ASP      2023-09-01 2023-09-01   2.92   2.94   0.68%  45,424,780   3.02   2.80 I Insert
AWC      2023-08-31 2023-09-01   4.42   4.26  -3.62% 386,017,008   4.42   4.06 B Update
BBL      2023-08-31 2023-09-01 171.00 169.00  -1.17%  36,592,608 171.00 166.50 O Update
BCH      2023-08-31 2023-09-01  19.80  19.00  -4.04% 360,006,216  19.80  18.60 I Update
BEM      2023-09-01 2023-09-01   8.65   8.70   0.58%  61,079,824   9.10   8.65 O Insert
BH       2023-09-01 2023-09-01 259.00 255.00  -1.54%  13,328,360 260.00 255.00 O Insert
CK       2023-08-31 2023-09-01  

In [27]:
os.chdir("C:\\users\\user\\onedrive\\a7\\Daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\Daily'

### cd\ruby\port_lite
### rails runner db/ins_sales.rb

In [30]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % today
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2023-09-01' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [31]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,AH,2023-08-30,2023-09-01,34.50,36.00,"22,142,968",36.00,34.00,B,SET,1.50,4.35%
1,AWC,2023-08-31,2023-09-01,4.42,4.26,"772,034,016",4.42,4.06,B,SET50,-0.16,-3.62%
2,DIF,2023-09-01,2023-09-01,9.45,9.35,"70,056,756",9.45,9.15,B,SET,-0.10,-1.06%
3,JASIF,2023-08-28,2023-09-01,6.95,6.75,"393,300,976",7.00,6.75,B,SET,-0.20,-2.88%
4,JMART,2023-09-01,2023-09-01,24.00,23.20,"173,996,992",24.00,21.20,B,SET50,-0.80,-3.33%
5,JMT,2023-08-31,2023-09-01,47.75,46.25,"50,545,772",47.75,45.50,B,SET50,-1.50,-3.14%
6,MCS,2023-08-31,2023-09-01,7.00,6.80,"4,170,368",7.00,6.00,B,SET,-0.20,-2.86%
7,NER,2023-09-01,2023-09-01,4.80,4.92,"162,020,712",4.92,4.56,B,SET,0.12,2.50%
8,ORI,2023-08-31,2023-09-01,10.80,10.50,"60,065,124",10.80,10.10,B,SET100,-0.30,-2.78%
9,PTTGC,2023-08-31,2023-09-01,37.75,37.00,"148,213,056",38.00,35.75,B,SET50,-0.75,-1.99%


In [33]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [35]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
42,SAPPE,2023-09-01,2023-09-01,100.00,91.00,"16,712,880",100.00,91.00,O,SET,-9.00,-9.00%
11,SINGER,2023-09-01,2023-09-01,14.90,14.10,"892,953,120",14.90,10.30,B,SET100,-0.80,-5.37%
45,TOA,2023-08-28,2023-09-01,28.00,26.50,"219,087,056",28.25,25.75,O,SET,-1.50,-5.36%


In [37]:
df_sort.query('percent >= 4.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,AH,2023-08-30,2023-09-01,34.50,36.00,"22,142,968",36.00,34.00,B,SET,1.50,4.35%
12,STA,2023-09-01,2023-09-01,14.60,15.30,"77,307,636",15.30,13.20,B,SET100,0.70,4.79%
